In [1]:
# Setting up the environment

import svelte_widget
import xgboost
import shap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

/Users/nicolazucchia/miniconda3/envs/svelte-widget/lib/python3.11/site-packages/shap/utils/_clustering.py:34: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/Users/nicolazucchia/miniconda3/envs/svelte-widget/lib/python3.11/site-packages/shap/utils/_clustering.py:53: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/Users/nicolazucchia/miniconda3/envs/svelt

In [2]:
# Upload and set up data

from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("diabetes_data.csv")

scaler = MinMaxScaler()

df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

X = df.drop(columns=['Outcome'])
y = df['Outcome']

In [3]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333,1.0
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667,0.0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1.0
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000,0.0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000,1.0
...,...,...,...,...,...,...,...,...,...
763,0.588235,0.507538,0.622951,0.484848,0.212766,0.490313,0.039710,0.700000,0.0
764,0.117647,0.613065,0.573770,0.272727,0.000000,0.548435,0.111870,0.100000,0.0
765,0.294118,0.608040,0.590164,0.232323,0.132388,0.390462,0.071307,0.150000,0.0
766,0.058824,0.633166,0.491803,0.000000,0.000000,0.448584,0.115713,0.433333,1.0


In [4]:
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Defining and training two Neural Networks

import tensorflow as tf
from tensorflow import keras

# Define the first neural network model
model_1 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_1.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Define the second neural network model
model_2 = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_2.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6656 - accuracy: 0.6401 - val_loss: 0.6541 - val_accuracy: 0.6429
Epoch 2/10
20/20 [==============================] - 0s 1ms/step - loss: 0.6419 - accuracy: 0.6564 - val_loss: 0.6405 - val_accuracy: 0.6429
Epoch 3/10
20/20 [==============================] - 0s 1ms/step - loss: 0.6255 - accuracy: 0.6612 - val_loss: 0.6262 - val_accuracy: 0.6558
Epoch 4/10
20/20 [==============================] - 0s 1ms/step - loss: 0.6100 - accuracy: 0.6645 - val_loss: 0.6142 - val_accuracy: 0.6753
Epoch 5/10
20/20 [==============================] - 0s 1ms/step - loss: 0.5960 - accuracy: 0.6824 - val_loss: 0.6027 - val_accuracy: 0.6364
Epoch 6/10
20/20 [==============================] - 0s 1ms/step - loss: 0.5821 - accuracy: 0.6971 - val_loss: 0.5924 - val_accuracy: 0.6234
Epoch 7/10
20/20 [==============================] - 0s 1ms/step - loss: 0.5675 - accuracy: 0.7036 - val_loss: 0.5817 - val_accuracy: 0.6753
Epoch 8/10
20/20 [==

In [6]:
z = svelte_widget.ExampleWidget(X_test, y_test, model_1.predict, model_2.predict)
z

TypeError: ExampleWidget.__init__() takes from 2 to 3 positional arguments but 5 were given